In [ ]:
using Markdown
using InteractiveUtils
using BenchmarkTools

# Reusing meaningful names meaningfully, good spelling, and structural lambda expressions.
## Gustavo Nunes Goretkin
### JuliaCon 2021
- discuss functions used as names
- demonstrate how multiple dispatch and generic functions enable good "spellings" and symbolic computation
- experimental package (FixArgs.jl) to showcase this pattern
- focus on the idea rather than the specific implementation, syntax, etc.

To concatenate two `AbstractVector`s, use `vcat`:

In [ ]:
vcat(1:5, 1:5)

Suppose we have a `Vector` of `Vector`s, and we want to concatenate all of the inner `Vector`s into one `Vector`.

In [ ]:
vs = [collect(1:10) for _ = 1:2000]; # `Vector` of `<:AbstractVector`s,  really.

Apply binary operation over a sequence using `reduce`:

In [ ]:
reduce(vcat, vs) |> string

Now let us do essentially the same computation, but instead of directly using `vcat`, we define a function that just calls `vcat`:

```julia
reduce((_1, _2) -> vcat(_1, _2), vs)
```

And let's time the two.

In [ ]:
@benchmark reduce(vcat, vs)

In [ ]:
@benchmark reduce((_1, _2) -> vcat(_1, _2), vs)

It is ~500× slower in this case.

Why? It is not because anonymous functions are slow (they aren't). Totally different code is running each case. (e.g. allocate output all at once)

In [ ]:
methods(reduce)

## Spelling
There is a reason that "spelling bees" were invented for English-language writing system.

Two key features:
- multiple dispatch
- the function is encoded in the type domain (i.e. each function is its own type)

enables defining special case for `reduce(vcat, ...)`.

Otherwise, would need name e.g. `reduce_vcat(...)`, or perhaps `flatten(...)`.

Personally, the first spelling is better. It combines existing and meaningful names instead of introducing a new ad-hoc name.

Note that running `reduce(vcat, ...)` might not even call `vcat` (as a function). `vcat` is used as a _name_.

More so than in other ecosystems, the Julia community aims to determine a generic meaning for a given function name. This is difficult and fundamental design work. It enables generic programming. It is good to reuse these names!

There are multiple converging motivations for the idea in this talk:

* Get extra value from careful function name / meaning pairs

* Generalize `Base.Fix1`/`Base.Fix2`

* Symbolic Computation / Lazy Computation

* Explore a combination of structural and of nominal types

## Demo of `Base.Fix2`
`==` is a two-argument function. `==(a, b)` is identical to `a == b`

In [ ]:
f1 = ==(50)

In [ ]:
f2 = x -> x == 50

`f1` and `f2` compute the same function. Some might say that they are different names for the same function.



And in this case, names matter!

In [ ]:
findfirst(f1, -1000:1000) # O(1)

In [ ]:
findfirst(f2, -1000:1000) # O(length(range))

In [ ]:
@which findfirst(f1, -1000:1000)

```julia
findfirst(p::Union{Fix2{typeof(isequal),T},Fix2{typeof(==),T}}, r::AbstractUnitRange) where {T<:Integer} =
    first(r) <= p.x <= last(r) ? 1+Int(p.x - first(r)) : nothing
```

This is why I love Julia. Imagine a plotting library that supports unevenly-spaced axis ticks.

Code written in terms of `findfirst` can support unevenly-spaced ticks, and still be (runtime) efficient when using evenly-spaced ticks.

The same code can handle a general case, without affecting performance in the special case!

The types `Fix1`/`Fix2` fix one argument of a two-argument function.


`FixArgs.jl` provides a generalization of `Fix1` and `Fix2` in a few ways:
1. A function of any positional arity can be used, and any number of its arguments can be bound, allowing the remaining arguments to be provided later.
2. A function can have its keyword arguments bound.
3. The function `x -> f(x, b)` is represented with types:
   - a [`Lambda`](https://goretkin.github.io/FixArgs.jl/dev/#FixArgs.Lambda) to represent function (`args -> body`)
   - a [`Call`](https://goretkin.github.io/FixArgs.jl/dev/#FixArgs.Call) to represent the function *call* (`f(...)`) in the body
   - a [`ArgPos`](https://goretkin.github.io/FixArgs.jl/dev/#FixArgs.ArgPos) to represent the `x` in the body of the lambda function

```julia
"""
    isapprox(x; kwargs...) / ≈(x; kwargs...)
Create a function that compares its argument to `x` using `≈`, i.e. a function equivalent to `y -> y ≈ x`.
The keyword arguments supported here are the same as those in the 2-argument `isapprox`.
"""
isapprox(y; kwargs...) = x -> isapprox(x, y; kwargs...)
```

The second generalization allows partial application with keyword arguments

All other docstrings in `Base` with "Create a function" use `Base.Fix2`, but here an anonymous function was used.

The third generalization is powerful, because it's effectively the [lambda calculus](https://en.wikipedia.org/wiki/Lambda_calculus).

What good is this power?

Would it ever be useful to fix all of the arguments of a function?

Consider the `/` (division) function.

If you fix its two arguments, that looks a lot like `Rational`.

In [ ]:
using FixArgs
half = @xquote 1 / 2

A macro helps create `Lambda` / `Call` / `ArgPos`, etc. objects

In [ ]:
half * half

Another macro helps define the types

In [ ]:
function Base.:*(a::(@xquoteT ::S / ::S), b::(@xquoteT ::S / ::S)) where S
    (n1, d1) = something.(Tuple(a.args))
    (n2, d2) = something.(Tuple(b.args))
    @xquote $(n1 * n2) / $(d1 * d2)
end

In [ ]:
half * half

Where does the connection between `Rational` and `/` happen in `Base`? [Here](https://github.com/JuliaLang/julia/blob/51f57406038df9f438edf911c8d1b59c9c1af4b1/base/rational.jl#L112).

    
```julia
AbstractFloat(x::Rational) = (float(x.num)/float(x.den))::AbstractFloat
```

In [ ]:
xeval(half)

## memory and code generation comparison

In [ ]:
sizeof(half)

In [ ]:
sizeof(1 // 2)

In [ ]:
reinterpret(Int, [half])

In [ ]:
@code_native xeval(half)

In [ ]:
@code_native float(1//2)

This is not a drop-in replacement for `Rational`, however, because

In [ ]:
1//2 isa Number

In [ ]:
(@xquote 1 / 2) isa Number

## Different than `Expr`
`/` in `:(/(x, y))` is not `Base.:/`, but just `:/`

In [ ]:
dump(:(x / y))

In [ ]:
let x = 1, y =2
    @xquote x / y
end

## Why?
Using this instead of `Base.Rational` seems pretty silly and confusing. Possible benefits:

* Some users want a "rational" type where the numerator and the denominator are not constrained to be the same type.

* a fixed-point number is one of these rational types where the denominator is "static" (a singleton type such that the numerical value is encoded in the type domain). Check the documentation for `FixArgs.jl` examples!

In [ ]:
MyQ0f7(x) = (@xquote $(Int8(x)) / 128::::S) # mark an argument as "static"
MyQ0f7(3)

### A combination of structural and nominal typing
Avoid the requirement to choose names like `Rational`, `num`, `den`.
The arguments can be distinguished by the role they play with respect to the function. Only the function needs a name.

## Existing patterns

`Base.Generator` could be a lazy representation of `map(f, itr)`

```julia
Base.Iterators.Filter(flt, itr)
```

could be replaced with

```julia
@xquote filter(flt, itr)
```

The type `Base.Iterators.Filter` could be an alias for

```julia
(@xquoteT filter(::F, ::I)) where {F, I}
```
to keep the existing symbolic "rules" such as [this one](https://github.com/JuliaLang/julia/blob/ef14131db321f8f5a815dd05a5385b5b27d87d8f/base/iterators.jl#L463):

```julia
reverse(f::Filter) = Filter(f.flt, reverse(f.itr))
```

[`Base.Iterators.Flatten(iterator_of_iterators)`](https://github.com/JuliaLang/julia/blob/ef14131db321f8f5a815dd05a5385b5b27d87d8f/base/iterators.jl#L1038-L1040) can be represented as `@xquote reduce(vcat, iterator_of_iterators)`

### `Base.literal_pow` 

In [ ]:
Meta.@lower x^2

Could instead lower to
```
%1 = @xquote x^(2::::S) # i.e. something roughly like `Call(^, x, Val(2))`
%2 = xeval(%1)
```

### Broadcasting
[Call](https://github.com/JuliaLang/julia/blob/d06c2a97be3f643d403c4069955e135823ff9fd0/base/broadcast.jl#L152-L173)

```julia
struct Broadcasted{Style<:Union{Nothing,BroadcastStyle}, Axes, F, Args<:Tuple} <: Base.AbstractBroadcasted
    f::F
    args::Args
    axes::Axes
end
```

(extra information `Style` and `axes`)

[xeval](https://github.com/JuliaLang/julia/blob/1910a7685a86d44041a98ff874f92e480fc44632/base/broadcast.jl#L904)

```julia
@inline materialize(bc::Broadcasted) = copy(instantiate(bc))
# [... `copyto!` methods ...]
```

[LazyArrays.jl](https://github.com/JuliaArrays/LazyArrays.jl)

[Call](https://github.com/JuliaArrays/LazyArrays.jl/blob/6b5cee2e8ed355a8e7590c3ae860faed4e816e19/src/lazyapplying.jl#L184-L187)

```julia
struct ApplyArray{T, N, F, Args<:Tuple} <: LazyArray{T,N}
    f::F
    args::Args
end

# [...]

const Vcat{T,N,I<:Tuple} = ApplyArray{T,N,typeof(vcat),I}

```

[xeval](https://github.com/JuliaArrays/LazyArrays.jl/blob/dff5924cd8b52c62a34cce16372381bb8a9e35cb/src/lazyconcat.jl#L20-L27)
```julia
function instantiate(A::Applied{DefaultApplyStyle,typeof(vcat)})
    isempty(A.args) && return A
    m = size(A.args[1],2)
    for k=2:length(A.args)
        size(A.args[k],2) == m || throw(ArgumentError("number of columns of each array must match (got $(map(x->size(x,2), A)))"))
    end
    Applied{DefaultApplyStyle}(A.f,map(instantiate,A.args))
end
```


[LazyStacks.jl](https://github.com/mcabbott/LazyStack.jl)

[Call](https://github.com/mcabbott/LazyStack.jl/blob/a02740adae65cd27acc0257a61a482fdebf2e2eb/src/LazyStack.jl#L82-L84)
```julia
struct Stacked{T,N,AT} <: AbstractArray{T,N}
    slices::AT
end
```


[xeval](https://github.com/mcabbott/LazyStack.jl/blob/a02740adae65cd27acc0257a61a482fdebf2e2eb/src/LazyStack.jl#L111)
```julia
Base.collect(x::Stacked{T,2,<:AbstractArray{<:AbstractArray{T,1}}}) where {T} = reduce(hcat, x.slices)
```

[LazySets.jl](https://github.com/JuliaReach/LazySets.jl)

[Call](https://github.com/JuliaReach/LazySets.jl/blob/4e3dc1668a0265f748fef4a5ced93c97337c2623/src/LazyOperations/Intersection.jl#L98-L108)

```julia
struct Intersection{N, S1<:LazySet{N}, S2<:LazySet{N}} <: LazySet{N}
    X::S1
    Y::S2
    cache::IntersectionCache
    # [...]
end
```

[xeval](https://github.com/JuliaReach/LazySets.jl/blob/4e3dc1668a0265f748fef4a5ced93c97337c2623/src/LazyOperations/Intersection.jl#L723-L725)
```julia
function concretize(cap::Intersection)
    return intersection(concretize(cap.X), concretize(cap.Y))
end
```

### Struct-of-Arrays and Array-of-Structs

In [ ]:
soa = (a=[1,2,3], b=[10, 20, 30])
aos = [(a=1, b=10), (a=2, b=20), (a=3, b=30)]

aos_eager = map(NamedTuple{(:a, :b)} ∘ tuple, soa.a, soa.b)

In [ ]:
soa_eager = NamedTuple{(:a, :b)}(tuple(getindex.(aos, :a), getindex.(aos, :b)))

In [ ]:
@xquote map(NamedTuple{(:a, :b)} ∘ tuple, soa.a, soa.b)

### Structural vs. Nominal

In [ ]:
@xquote x -> x # `identity` could be an alias for this

In [ ]:
x -> x

In [ ]:
x -> x # gets a different name, even though structurally identical.

September 2020:
The composition operator `∘` now returns a [`Base.ComposedFunction`](https://github.com/jw3126/julia/blob/37908d66492f3cf95759f8e0e7e3d2bd6d038c0c/base/operators.jl#L909-L914) instead of an anonymous function ([#37517]).

```julia
struct ComposedFunction{O,I} <: Function
    outer::O
    inner::I
end
```

## Different niche from Computer Algebra System

[SymbolicUtils.Term](https://github.com/JuliaSymbolics/SymbolicUtils.jl/blob/c3081cdbf59b8e07ed3a757d8e9eb8bdbc9cad6e/src/types.jl#L323-L328) intentionally does not dispatch on function or argument types:

```julia
struct Term{T, M} <: Symbolic{T}
    f::Any
    arguments::Any
    metadata::M
    hash::Ref{UInt} # hash cache
end
```


## Taking the idea too far?

In [ ]:
typeof(im)

`im` is defined in terms of `Complex`. Going the other way

In [ ]:
𝗶 = @xquote sqrt((-1)::::S)

In [ ]:
xeval(𝗶)

In [ ]:
c = @xquote 1 + 2*𝗶

In [ ]:
reinterpret(Int, [c])

Note that
- `@xquote 1 + 2*𝗶`
- `@xquote 2*𝗶 + 1`
- `@xquote 𝗶*2 + 1`
are all different types.

## Downsides
- code reuse increases coupling
- burden on the compiler
- huge types
- method ambiguities

# Thanks!